Esse dataset é conhecido como Census Income ou Adult dataset, e foi extraído do banco de dados do Censo dos EUA de 1994. O objetivo principal do dataset é prever se uma pessoa ganha mais de 50 mil dólares por ano com base em suas características demográficas e socioeconômicas. Aqui estão os principais detalhes sobre o dataset:
Características do Dataset:

- Tarefas associadas: Classificação. A tarefa é prever se o rendimento de uma pessoa excede 50 mil dólares por ano.
Número de instâncias (linhas): 48.842 pessoas.
Número de features (colunas): 14, sendo que algumas são categóricas (como estado civil e ocupação) e outras são numéricas (como idade e horas trabalhadas por semana).
Valores ausentes: Algumas colunas têm valores ausentes (exemplo: workclass e occupation).


Objetivo:

O modelo deve prever se uma pessoa ganha mais de 50 mil dólares anuais (classe >50K) ou menos/igual a isso (classe <=50K), utilizando as 14 características fornecidas.
Explicação das Variáveis:

age: Idade da pessoa (inteiro).
        Tipo: Numérico
        Valores ausentes: Não

workclass: Categoria de emprego da pessoa (ex.: privado, governo local, etc.).
        Tipo: Categórico
        Valores ausentes: Sim (alguns registros não têm essa informação)

fnlwgt: Peso final da amostra. Este valor indica quantas pessoas na população original são representadas por essa instância.
        Tipo: Numérico
        Valores ausentes: Não

education: Nível de educação da pessoa (ex.: bacharelado, ensino médio, etc.).
        Tipo: Categórico
        Valores ausentes: Não

education-num: Nível de escolaridade representado por um número (1-16).
        Tipo: Numérico
        Valores ausentes: Não

marital-status: Estado civil (ex.: casado, divorciado, solteiro, etc.).
        Tipo: Categórico
        Valores ausentes: Não

occupation: Tipo de ocupação (ex.: gerência executiva, vendas, suporte técnico, etc.).
        Tipo: Categórico
        Valores ausentes: Sim

relationship: Papel dentro da família (ex.: esposa, marido, filho, etc.).
        Tipo: Categórico
        Valores ausentes: Não

race: Raça (ex.: branca, asiática, negra, etc.).
        Tipo: Categórico
        Valores ausentes: Não

sex: Gênero (feminino ou masculino).
        Tipo: Binário
        Valores ausentes: Não

capital-gain: Ganho de capital (numérico).
        Tipo: Numérico
        Valores ausentes: Não

capital-loss: Perda de capital (numérico).
        Tipo: Numérico
        Valores ausentes: Não

hours-per-week: Horas trabalhadas por semana.
        Tipo: Numérico
        Valores ausentes: Não

native-country: País de origem (ex.: Estados Unidos, Canadá, etc.).
        Tipo: Categórico
        Valores ausentes: Sim

Pontos Importantes:

- Valores ausentes: Algumas variáveis como workclass, occupation, e native-country têm valores ausentes, o que exige tratamento na preparação dos dados.
Problema de classificação: A variável de saída (target) é se a pessoa ganha mais de 50K (>50K) ou menos/igual a 50K (<=50K).
Características categóricas e numéricas: Algumas variáveis são categóricas, o que requer técnicas como one-hot encoding para usá-las em algoritmos de aprendizado de máquina.

Aplicações:

- Esse dataset é usado para construir modelos preditivos de classificação, onde as variáveis são usadas para prever a classe de rendimento (>50K ou <=50K). Ele é amplamente utilizado em tutoriais e estudos de classificação em machine learning.

In [12]:
# from google.colab import drive
# drive.mount('/content/drive')

In [13]:
#!pip install aequitas

In [14]:
#!pip install keras-tuner

In [15]:
# Importações de bibliotecas necessárias para o tratamento de dados
import math
import pandas as pd
import numpy as np

# Importações de bibliotecas para manipulação de dados e modelos
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle # Embaralhar os dados

# Normalização de dados
from sklearn.preprocessing import OneHotEncoder # Converte categorias em colunas binárias (0 ou 1)
from sklearn.preprocessing import LabelEncoder   # Converte categorias em números
from sklearn.preprocessing import StandardScaler # Normalização usando Z-score

# Importações para estatísticas e testes estatísticos
from statsmodels.stats.proportion import proportions_ztest
from scipy.stats import norm

# Métricas de avaliação de modelos
from sklearn.metrics import accuracy_score, classification_report,accuracy_score
from sklearn.metrics import confusion_matrix, roc_auc_score, roc_curve


# Modelos de machine learning do scikit-learn
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.linear_model import LogisticRegression

# Importações de bibliotecas para redes neurais usando TensorFlow/Keras
import tensorflow as tf
from tensorflow.keras.layers import Dense, LeakyReLU, BatchNormalization, Reshape, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import layers, Input
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import regularizers
import keras_tuner as kt





# Bibliotecas para gráficos
import matplotlib.pyplot as plt
import seaborn as sns

from aequitas import Audit
from aequitas.plotting import Plot
from aequitas.group import Group
from aequitas.audit import Audit


# **Tratamento de dados**

In [16]:
def grafico_matriz_confusao(conf_matrix,accuracy, titulo):

    conf_matrix_percentage = conf_matrix.astype('float') / conf_matrix.sum(axis=1)[:, np.newaxis] * 100

    # Função para adicionar o símbolo de porcentagem aos valores na matriz de confusão
    labels = [f'{value:.2f}%' for value in conf_matrix_percentage.flatten()]
    labels = np.array(labels).reshape(conf_matrix.shape)

    # Converter a acurácia para porcentagem
    accuracy_percentage = accuracy * 100

    # Plot da matriz de confusão com porcentagens e a acurácia em porcentagem
    plt.figure(figsize=(8, 6))
    sns.heatmap(conf_matrix_percentage, annot=labels, fmt='', cmap='Blues', cbar=False, annot_kws={"size": 16})
    plt.title(f'Matriz de Confusão (Porcentagem)\n{titulo}\nAcurácia: {accuracy_percentage:.2f}%', size=16)
    plt.xlabel('Predição', size=14)
    plt.ylabel('Real', size=14)
    return plt.show()



def grafico_curva_roc(fpr, tpr,roc_auc, titulo):
    # Plot da curva ROC

    plt.figure()
    plt.plot(fpr, tpr, color='blue', lw=2, label='Curva ROC (área = %0.2f)' % roc_auc)
    plt.plot([0, 1], [0, 1], color='gray', lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('Taxa de Falsos Positivos')
    plt.ylabel('Taxa de Verdadeiros Positivos')
    plt.title(f'Curva ROC {titulo}')
    plt.legend(loc="lower right")
    print(f"ROC AUC: {roc_auc}")
    return plt.show()

def grafico_metricas_sensiveis(y_pred):
    # Preparar o DataFrame para o Aequitas
    aequitas_df = test_data_adult[['sex', 'race']].copy()
    aequitas_df['label'] = test_data_adult['income']  # Renomeia a coluna de rótulo para 'label_value'
    aequitas_df['score'] = y_pred  # Adiciona as previsões do modelo

    # Inicializando o Audit sem passar o DataFrame
    audit = Audit(aequitas_df)

    # Gera o gráfico de summary_plot para as métricas desejadas
    audit.audit()
    return audit.summary_plot(["tpr", "fpr", "pprev"])


In [17]:
# # train_url = '/content/adult.data'
# train_url = "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data"
# link_index = '/content/Index'

# test_url = "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.test"

# # link_informacoes = '/content/adult.names'
# link_informacoes = 'https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.names'
# # test_url = '/content/adult.test'

# # Nome das colunas do dataset
# column_names = [
#     'age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status',
#     'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss',
#     'hours-per-week', 'native-country', 'income'
# ]

# test_data_adult = pd.read_csv(test_url, header=0, names=column_names, na_values=" ?", sep=',\s', engine='python')
# train_data_adult = pd.read_csv(train_url, header=None, names=column_names, na_values=" ?", sep=',\s', engine='python')

In [18]:
# # Substituir strings vazias por NaN e remover linhas com valores nulos
# train_data_adult.dropna(inplace=True)
# test_data_adult.dropna(inplace=True)


# # Codificar a variável alvo ('income') transformando em binarios
# train_data_adult['income'] = train_data_adult['income'].apply(lambda x: 1 if x == '>50K' else 0)
# test_data_adult['income'] = test_data_adult['income'].apply(lambda x: 1 if x == '>50K.' else 0)

# # Codificar a variável sex transformando em binarios
# train_data_adult['sex'] = train_data_adult['sex'].apply(lambda x: 1 if x == 'Male' else 0)
# test_data_adult['sex'] = test_data_adult['sex'].apply(lambda x: 1 if x == 'Male' else 0)
# # Codificação de variáveis categóricas para binarios
# categorical_cols = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'native-country']

# for col in categorical_cols:
#     label_encoder = LabelEncoder()
#     train_data_adult[col] = label_encoder.fit_transform(train_data_adult[col])
#     test_data_adult[col] = label_encoder.transform(test_data_adult[col])

# # scaler = StandardScaler()
# # encoded_train_features = scaler.fit_transform(train_data_adult[categorical_cols])
# # encoded_test_features = scaler.transform(test_data_adult[categorical_cols])

In [19]:
# train_data_adult

In [20]:
# # excluindo dados
# # Substituir strings vazias por NaN
# train_data_adult.replace('', np.nan, inplace=True)

# # Remover linhas com valores nulos
# train_data_adult_cleaned = train_data_adult.dropna()

In [21]:
# dados_sensiveis_train = pd.DataFrame()

# dados_sensiveis_train['Amer-Indian-Eskimo'] = train_data_adult.iloc[:,8].apply(lambda x: 1 if x == 'Amer-Indian-Eskimo' else 0)
# dados_sensiveis_train['Asian-Pac-Islander'] = train_data_adult.iloc[:,8].apply(lambda x: 1 if x == 'Asian-Pac-Islander' else 0)
# dados_sensiveis_train['Black'] = train_data_adult.iloc[:,8].apply(lambda x: 1 if x == 'Black' else 0)
# dados_sensiveis_train['Other'] = train_data_adult.iloc[:,8].apply(lambda x: 1 if x == 'Other' else 0)
# dados_sensiveis_train['White'] = train_data_adult.iloc[:,8].apply(lambda x: 1 if x == 'White' else 0)
# dados_sensiveis_train['Genero'] = train_data_adult.iloc[:,9].apply(lambda x: 1 if x == 'Male' else 0)

# dados_sensiveis_train['income_binary'] = train_data_adult.iloc[:,-1]

In [22]:
# dados_sensiveis_test = pd.DataFrame()

# dados_sensiveis_test['Amer-Indian-Eskimo'] = test_data_adult.iloc[:,8].apply(lambda x: 1 if x == 'Amer-Indian-Eskimo' else 0)
# dados_sensiveis_test['Asian-Pac-Islander'] = test_data_adult.iloc[:,8].apply(lambda x: 1 if x == 'Asian-Pac-Islander' else 0)
# dados_sensiveis_test['Black'] = test_data_adult.iloc[:,8].apply(lambda x: 1 if x == 'Black' else 0)
# dados_sensiveis_test['Other'] = test_data_adult.iloc[:,8].apply(lambda x: 1 if x == 'Other' else 0)
# dados_sensiveis_test['White'] = test_data_adult.iloc[:,8].apply(lambda x: 1 if x == 'White' else 0)
# dados_sensiveis_test['Genero'] = test_data_adult.iloc[:,9].apply(lambda x: 1 if x == 'Male' else 0)

# dados_sensiveis_test['income_binary'] = test_data_adult.iloc[:,-1]

In [23]:
# # Excluindo a coluna 'B'
# train_data_adult = train_data_adult.drop('race', axis=1)
# #train_data_adult = train_data_adult.drop('B', axis=1)
# test_data_adult = test_data_adult.drop('race', axis=1)
# #test_data_adult = test_data_adult.drop('B', axis=1)

In [24]:

# # Concatenação das colunas codificadas com as numéricas
# numeric_cols = ['age', 'fnlwgt', 'education-num', 'capital-gain', 'capital-loss', 'hours-per-week']
# X_train = pd.concat([dados_sensiveis_train, train_data_adult[numeric_cols].reset_index(drop=True)], axis=1)
# X_test = pd.concat([dados_sensiveis_test, test_data_adult[numeric_cols].reset_index(drop=True)], axis=1)

# # Converter os nomes das colunas para strings
# X_train.columns = X_train.columns.astype(str)
# X_test.columns = X_test.columns.astype(str)

# # Variável alvo
# y_train = train_data_adult['income']
# y_test = test_data_adult['income']

# # Dividir o conjunto de treinamento em treino e validação
# # X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)



# #______________________________________________________________
# from imblearn.over_sampling import SMOTE

# smote = SMOTE()


# # Rebalanceamento
# X_resampled, y_resampled = smote.fit_resample(X_train, y_train)

# # Dividir o conjunto de treinamento em treino e validação
# X_train, X_val, y_train, y_val = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)


# #______________________________________________________________

# # Padronizar as variáveis numéricas
# scaler = StandardScaler()
# X_train = scaler.fit_transform(X_train)
# X_val = scaler.transform(X_val)
# X_test = scaler.transform(X_test)

In [25]:
# # URLs dos datasets de treinamento e teste
train_url = "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data"
test_url = "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.test"

# Nome das colunas do dataset
column_names = [
    'age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status',
    'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss',
    'hours-per-week', 'native-country', 'income'
]

# Carregar os dados
train_data_adult = pd.read_csv(train_url, header=None, names=column_names, na_values=" ?", sep=',\s', engine='python')
test_data_adult = pd.read_csv(test_url, header=0, names=column_names, na_values=" ?", sep=',\s', engine='python')

train_data_adult.dropna(inplace=True)
test_data_adult.dropna(inplace=True)

# Codificar a variável alvo ('income')
train_data_adult['income'] = train_data_adult['income'].apply(lambda x: 1 if x.strip() == '>50K' else 0)
test_data_adult['income'] = test_data_adult['income'].apply(lambda x: 1 if x.strip() == '>50K.' else 0)

# Codificação das variáveis categóricas
categorical_cols = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'native-country']
encoder = OneHotEncoder(drop='first', sparse_output=False)
encoded_train_features = encoder.fit_transform(train_data_adult[categorical_cols])
encoded_test_features = encoder.transform(test_data_adult[categorical_cols])



# Concatenação das colunas codificadas com as numéricas
numeric_cols = ['age', 'fnlwgt', 'education-num', 'capital-gain', 'capital-loss', 'hours-per-week']
X_train = pd.concat([pd.DataFrame(encoded_train_features), train_data_adult[numeric_cols].reset_index(drop=True)], axis=1)
X_test = pd.concat([pd.DataFrame(encoded_test_features), test_data_adult[numeric_cols].reset_index(drop=True)], axis=1)

# Converter os nomes das colunas para strings
X_train.columns = X_train.columns.astype(str)
X_test.columns = X_test.columns.astype(str)

# Variável alvo
y_train = train_data_adult['income']
y_test = test_data_adult['income']

# Dividir o conjunto de treinamento em treino e validação
# X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)



#______________________________________________________________
from imblearn.over_sampling import SMOTE

smote = SMOTE()


# Rebalanceamento
X_resampled, y_resampled = smote.fit_resample(X_train, y_train)

# Dividir o conjunto de treinamento em treino e validação
X_train, X_val, y_train, y_val = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)


#______________________________________________________________

# Padronizar as variáveis numéricas
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

In [26]:
# # O autoencoder é uma arquitetura de rede neural usada principalmente para aprendizado não supervisionado e compressão de dados,
# # onde o objetivo é aprender uma representação comprimida (codificada) dos dados de entrada e, em seguida, reconstruir os dados originais.

# def build_autoencoder(hp):
#     input_shape = 100

#     input_layer = layers.Input(shape=(input_shape,))
#     encoded = layers.Dense(hp.Int('units1', min_value=32, max_value=128, step=32), activation='relu')(input_layer)
#     encoded = layers.Dense(hp.Int('units2', min_value=16, max_value=64, step=16), activation='relu')(encoded)

#     decoded = layers.Dense(hp.Int('units3', min_value=32, max_value=128, step=32), activation='relu')(encoded)
#     decoded = layers.Dense(input_shape, activation='sigmoid')(decoded)

#     autoencoder = tf.keras.Model(inputs=input_layer, outputs=decoded)
#     autoencoder.compile(optimizer=tf.keras.optimizers.Adam(hp.Float('learning_rate', 1e-4, 1e-2, sampling='LOG')), loss='mse')
#     return autoencoder

# def tune_autoencoder(x_train):
#     tuner = kt.Hyperband(
#         build_autoencoder,
#         objective='val_loss',
#         max_epochs=20,
#         factor=3,
#         directory='autoencoder_tuning',
#         project_name='autoencoder'
#     )

#     tuner.search(X_train, X_train, epochs=10, validation_split=0.2)
#     best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
#     # Print hyperparameters in a more readable format
#     print("Best hyperparameters:")
#     print(f"Units in the first layer: {best_hps.get('units1')}")
#     print(f"Units in the second layer: {best_hps.get('units2')}")
#     print(f"Units in the third layer: {best_hps.get('units3')}")
#     print(f"Learning rate: {best_hps.get('learning_rate')}")

#     return best_hps


# tune_autoencoder(X_train)

In [27]:
# best_hps = tune_autoencoder(X_train)

In [28]:
# Verifique as formas após o SMOTE
print(X_resampled.shape, y_resampled.shape)


(49440, 100) (49440,)


In [29]:
# Definir a arquitetura do Autoencoder
input_shape = X_train.shape[1]

def build_autoencoder(input_shape, units1, units2, units3, learning_rate):
    input_layer = tf.keras.layers.Input(shape=(input_shape,))
    encoded = tf.keras.layers.Dense(units1, activation='relu')(input_layer)
    encoded = tf.keras.layers.Dense(units2, activation='relu')(encoded)
    decoded = tf.keras.layers.Dense(units3, activation='relu')(encoded)
    decoded = tf.keras.layers.Dense(input_shape, activation='sigmoid')(decoded)

    autoencoder = tf.keras.Model(inputs=input_layer, outputs=decoded)
    autoencoder.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate), loss='mse')
    return autoencoder


In [30]:
# Treinamento do Autoencoder
# Classificação Baseada em Custo
# Para ajustar o peso das classes em redes neurais, você pode usar o argumento class_weight durante o treinamento.
# Isso penaliza mais os erros nas classes minoritárias. Aqui está um exemplo básico usando o Keras:

# Instanciando o autoencoder
# autoencoder = build_autoencoder(
#     input_shape=input_shape,
#     units1=best_hps.get('units1'),  # Aqui você estava repetindo `input_shape` por engano
#     units2=best_hps.get('units2'),
#     units3=best_hps.get('units3'),
#     learning_rate=best_hps.get('learning_rate')
# )


autoencoder = build_autoencoder(13,  96,   48,  96,   0.00012225501914742633)

# Treinamento com pesos de classe
autoencoder.fit(X_train, X_train, epochs=50, batch_size=128, shuffle=True,
                validation_data=(X_val, X_val))



Epoch 1/50


ValueError: Input 0 of layer "functional" is incompatible with the layer: expected shape=(None, 13), found shape=(128, 100)

In [ ]:
X_val.shape

In [ ]:
X_train.shape

In [ ]:
# a codificação do autoencoder é usada para obter uma representação reduzida dos dados de entrada.
X_train_encoded = autoencoder.predict(X_train)
X_val_encoded = autoencoder.predict(X_val)
X_test_encoded = autoencoder.predict(X_test)

In [ ]:
# Projetada para construir um classificador com hiperparâmetros ajustáveis, que são fornecidos pelo objeto hp (HyperParameters).
# Isso é usado em conjunto com o Keras Tuner para realizar uma busca automatizada pelos melhores hiperparâmetros para o modelo.
def build_classifier(hp):
    input_shape = X_train.shape[1]

    input_layer = tf.keras.layers.Input(shape=(input_shape,))

    # Hiperparâmetros para o número de unidades nas camadas ocultas
    units1 = hp.Int('units1', min_value=32, max_value=128, step=32)
    units2 = hp.Int('units2', min_value=16, max_value=64, step=16)

    # Hiperparâmetro para a taxa de dropout
    dropout_rate = hp.Float('dropout_rate', min_value=0.3, max_value=0.7, step=0.1)

    hidden = tf.keras.layers.Dense(units1, activation='relu',
                                   kernel_regularizer=regularizers.l2(0.001))(input_layer)
    hidden = tf.keras.layers.Dropout(dropout_rate)(hidden)
    hidden = tf.keras.layers.Dense(units2, activation='relu',
                                   kernel_regularizer=regularizers.l2(0.001))(hidden)
    output_layer = tf.keras.layers.Dense(1, activation='sigmoid')(hidden)

    classifier = tf.keras.Model(inputs=input_layer, outputs=output_layer)
    classifier.compile(optimizer=tf.keras.optimizers.Adam(), loss='binary_crossentropy', metrics=['accuracy'])
    return classifier

In [ ]:
#  Keras Tuner para realizar a busca por hiperparâmetros e encontrar a melhor
# configuração para o classificador definido pela função build_classifier
def tune_classifier(x_train, y_train):
    tuner = kt.Hyperband(
        build_classifier,
        objective='val_accuracy',
        max_epochs=20,
        factor=3,
        directory='classifier_tuning',
        project_name='classifier'
    )

    tuner.search(x_train, y_train, epochs=20, validation_split=0.2)
    best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

    return best_hps


In [ ]:
best_hps = tune_classifier(X_train, y_train)


# Imprimir os melhores hiperparâmetros
print("Melhores Hiperparâmetros Encontrados:")
print(best_hps)

# Aplicar os melhores hiperparâmetros encontrados
input_shape = X_train.shape[1]
classifier = build_classifier(best_hps)

# Early Stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Treinar o modelo com os melhores hiperparâmetros
# history = classifier.fit(X_train, y_train, epochs=20, validation_split=0.2)
history_classifier = classifier.fit(X_train_encoded, y_train, epochs=20, batch_size=128, validation_data=(X_val_encoded, y_val), callbacks=[early_stopping])

In [ ]:
y_pred_income = autoencoder.predict(X_test_encoded)
y_pred_income = (y_pred_income > 0.5).astype(int)
np.unique(y_pred_income)

In [ ]:
X_test_encoded.shape

In [ ]:
y_test.shape

In [ ]:
# Previsão do classificador no conjunto de teste

#y_pred_income = autoencoder.predict(X_test_encoded)
y_pred_income = classifier.predict(X_test_encoded)
y_pred_income = (y_pred_income > 0.5).astype(int).reshape(-1,1)

# Avaliação do classificador para prever 'income'
conf_matrix_income = confusion_matrix(y_test, y_pred_income)

# Calcular porcentagens
conf_matrix_percentage = conf_matrix_income.astype('float') / conf_matrix_income.sum(axis=1)[:, np.newaxis] * 100
labels = [f"{value:.2f}%" for value in conf_matrix_percentage.flatten()]
labels = np.array(labels).reshape(conf_matrix_income.shape)

# Calcular a Curva ROC
fpr, tpr, _ = roc_curve(y_test, classifier.predict(X_test_encoded))
roc_auc = roc_auc_score(y_test, classifier.predict(X_test_encoded))


# Acurácia (em porcentagem)
accuracy_income = accuracy_score(y_test, y_pred_income)
accuracy_income

In [ ]:

print(classification_report(y_test, y_pred_income))

In [ ]:
# # URLs dos datasets de treinamento e teste
train_url = "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data"
test_url = "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.test"

# Nome das colunas do dataset
column_names = [
    'age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status',
    'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss',
    'hours-per-week', 'native-country', 'income'
]

# Carregar os dados
train_data_adult = pd.read_csv(train_url, header=None, names=column_names, na_values=" ?", sep=',\s', engine='python')
test_data_adult = pd.read_csv(test_url, header=0, names=column_names, na_values=" ?", sep=',\s', engine='python')

train_data_adult.dropna(inplace=True)
test_data_adult.dropna(inplace=True)

# Codificar a variável alvo ('income')
#train_data_adult['income'] = train_data_adult['income'].apply(lambda x: 1 if x.strip() == '>50K' else 0)
test_data_adult['income'] = test_data_adult['income'].apply(lambda x: 1 if x.strip() == '>50K.' else 0)


def grafico_metricas_sensiveis(y_pred):
    # Preparar o DataFrame para o Aequitas
    aequitas_df = test_data_adult[['sex', 'race']].copy()
    aequitas_df['label'] = test_data_adult['income']  # Renomeia a coluna de rótulo para 'label_value'
    aequitas_df['score'] = y_pred  # Adiciona as previsões do modelo

    # Inicializando o Audit sem passar o DataFrame
    audit = Audit(aequitas_df)

    # Gera o gráfico de summary_plot para as métricas desejadas
    audit.audit()
    return audit.summary_plot(["tpr", "fpr", "pprev"])

In [ ]:
grafico_metricas_sensiveis(y_pred_income)

In [ ]:
#grafico_matriz_confusao(conf_matrix_income,accuracy_income,'Modelo FAN')

In [ ]:
# Plotar a curva de loss (erro)
# plt.plot(history_classifier.history['loss'], label='Loss Treinamento')
# plt.plot(history_classifier.history['val_loss'], label='Loss Validação')
# plt.title('Curva de Loss do Classificador')
# plt.xlabel('Épocas')
# plt.ylabel('Loss')
# plt.legend()
# plt.show()


In [ ]:
# # Salvar o autoencoder no formato nativo Keras
# autoencoder.save('autoencoder_model.keras')

# # Salvar o classificador no formato nativo Keras
# classifier.save('classifier_model.keras')



In [ ]:
# # Carregar o autoencoder
# autoencoder_loaded = load_model('/content/autoencoder_model.keras')

# # Carregar o classificador
# classifier_loaded = load_model('/content/classifier_model.keras')


In [ ]:
# # Previsão com o autoencoder carregado
# X_test_encoded = autoencoder_loaded.predict(X_test)

# # Previsão com o classificador carregado
# y_pred_income = classifier_loaded.predict(X_test_encoded)
# y_pred_income = (y_pred_income > 0.5).astype(int)

# # Avaliação do classificador
# accuracy_income = accuracy_score(y_test, y_pred_income) * 100
# print(f'Acurácia: {accuracy_income:.2f}%')


# # history_classifier = classifier_loaded.fit(X_train_encoded, y_train, epochs=10, batch_size=128,
# #                                            validation_data=(X_val_encoded, y_val))